# setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

sklearn version:  1.0.2
TF version:  2.8.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


# Get the data

In [2]:
#작성자 전민재, hsh
import csv
def load_Emist(exsitNumpy=False, needTranspose=True):
    if (exsitNumpy == False):
        #"./emnist-byclass-test.csv"
        #train-set
        #"공용/datasets/Emnist/" 위치에 csv 저장 
        csv_train_data_file = open("../공용/datasets/Emnist/train_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        csv_train_label_file = open("../공용/datasets/Emnist/train_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
    
        
        f_train = csv.reader(csv_train_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        l_train=csv.reader(csv_train_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        #valid-set
    
        csv_valid_data_file = open("../공용/datasets/Emnist/valid_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        csv_valid_label_file = open("../공용/datasets/Emnist/valid_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        
        f_valid = csv.reader(csv_valid_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        l_valid = csv.reader(csv_valid_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        X_train=[]
        y_train=[]
        X_valid=[]
        y_valid=[]
        for i, row in enumerate(f_train):
            #행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx]=int(char)
            #train
            #data 추가    
            X_train.append(row)
            
        for i, row in enumerate(f_valid):
            #행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx]=int(char)
            #valid
            #data 추가    
            X_valid.append(row)


        for i, row in enumerate(l_train):

            #train
            #label 추가
            y_train.append(int(row[0]))
            
        for i, row in enumerate(l_valid):

            #valid
            #label 추가
            y_valid.append(int(row[0]))
            
            
        

        #train
        X_train=np.array(X_train,dtype=np.uint8)
        X_train=X_train.reshape(-1,28,28)
        #valid
        X_valid=np.array(X_valid,dtype=np.uint8)
        X_valid=X_valid.reshape(-1,28,28)
        
        csv_train_data_file.close()
        csv_train_label_file.close()
        csv_valid_data_file.close()
        csv_valid_label_file.close()
        
        #kaggle dataset이 시계반대방향으로 90도 회전 되있고 상하 반전 되어있음
        def rotate_90(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[c][N-1-r] = m[r][c]
            return np.array(ret,dtype=np.uint8)

        def vreflect(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[r][c] = m[r][N-1-c]
            return np.array(ret,dtype=np.uint8)
        
        if needTranspose == True:
            #train
            #회전
            for idx,i in enumerate(X_train):
                X_train[idx]=rotate_90(i)
            #상하반전
            for idx,i in enumerate(X_train):
                X_train[idx]=vreflect(i)

            #valid
            #회전
            for idx,i in enumerate(X_valid):
                X_valid[idx]=rotate_90(i)
            #상하반전
            for idx,i in enumerate(X_valid):
                X_valid[idx]=vreflect(i)

       
        
        
        #저장용
        #train
        #(533917,28,28)
        
        np.save('./X_train',np.array(X_train,dtype=np.uint8))
        np.save('./y_train',np.array(y_train,dtype=np.uint8))
        #valid
        np.save('./X_valid',np.array(X_valid,dtype=np.uint8))
        np.save('./y_valid',np.array(y_valid,dtype=np.uint8))
        
   
    X_train=np.load('./X_train.npy')
    y_train=np.load('./y_train.npy')
    X_valid=np.load('./X_valid.npy')
    y_valid=np.load('./y_valid.npy')
    
    return X_train, y_train, X_valid, y_valid


In [3]:
#황성현
#npy 파일이 존재할 경우 exsitNumpy = True, 업으면 False
X_train, y_train, X_valid, y_valid = load_Emist(exsitNumpy=True)
X_train.shape,X_train.dtype

((533917, 28, 28), dtype('int32'))

In [4]:
#황성현
X_train=tf.reshape(X_train,[-1,28,28,1])
X_valid=tf.reshape(X_valid,[-1,28,28,1])

In [5]:
#황성현
#data, batch size 성정
train_size=len(X_train)
valid_size=len(X_valid)
batch_size=32
train_size

533917

In [6]:
#황성현,jmj
X_train.shape,X_valid.shape,X_train.dtype

(TensorShape([533917, 28, 28, 1]), TensorShape([164015, 28, 28, 1]), tf.int32)

In [7]:
#황성현, jmj
X_train_resize=np.uint8(tf.image.resize(X_train, [32, 32]))
X_valid_resize=np.uint8(tf.image.resize(X_valid, [32, 32]))


In [8]:
#황성현
X_train_resize.shape,X_valid_resize.dtype,X_valid_resize.shape

((533917, 32, 32, 1), dtype('uint8'), (164015, 32, 32, 1))

# train the model

# SMV2L without argumentation when learning rate is 0.002

In [9]:
#황성현
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, 
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME", activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [10]:
#황성현
n_classes=62
smv2_l=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 32, 32, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)     

In [11]:
smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

In [14]:
batch_size=32

In [16]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_lr_002_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_lr_002_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
16685/16685 [==============================] - 1629s 97ms/step - loss: 0.7771 - accuracy: 0.7560 - val_loss: 0.4884 - val_accuracy: 0.8312
Epoch 2/300
16685/16685 [==============================] - 1598s 96ms/step - loss: 0.4703 - accuracy: 0.8344 - val_loss: 0.4338 - val_accuracy: 0.8459
Epoch 3/300
16685/16685 [==============================] - 1632s 98ms/step - loss: 0.4336 - accuracy: 0.8448 - val_loss: 0.4101 - val_accuracy: 0.8476
Epoch 4/300
16685/16685 [==============================] - 1619s 97ms/step - loss: 0.4154 - accuracy: 0.8501 - val_loss: 0.3964 - val_accuracy: 0.8539
Epoch 5/300
16685/16685 [==============================] - 1658s 99ms/step - loss: 0.4041 - accuracy: 0.8532 - val_loss: 0.3910 - val_accuracy: 0.8577
Epoch 6/300
16685/16685 [==============================] - 1716s 103ms/step - loss: 0.3959 - accuracy: 0.8556 - val_loss: 0.3894 - val_accuracy: 0.8571
Epoch 7/300
16685/16685 [==============================] - 1714s 103ms/step - loss: 0.3891 - 

# smv2p

In [17]:
n_classes=62
smv2_p=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_p.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 32)
IRB4 : batch_input_shape = (None, 4, 4, 64)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 32)   

In [18]:
smv2_p.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

In [19]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2P_lr_002_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2P_nadam_lr_002_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_p.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
16685/16685 [==============================] - 1788s 107ms/step - loss: 0.7399 - accuracy: 0.7650 - val_loss: 0.4794 - val_accuracy: 0.8323
Epoch 2/300
16685/16685 [==============================] - 1774s 106ms/step - loss: 0.4706 - accuracy: 0.8346 - val_loss: 0.4347 - val_accuracy: 0.8458
Epoch 3/300
16685/16685 [==============================] - 1771s 106ms/step - loss: 0.4386 - accuracy: 0.8434 - val_loss: 0.4116 - val_accuracy: 0.8491
Epoch 4/300
16685/16685 [==============================] - 1767s 106ms/step - loss: 0.4229 - accuracy: 0.8485 - val_loss: 0.4022 - val_accuracy: 0.8543
Epoch 5/300
16685/16685 [==============================] - 1773s 106ms/step - loss: 0.4119 - accuracy: 0.8510 - val_loss: 0.4018 - val_accuracy: 0.8552
Epoch 6/300
16685/16685 [==============================] - 1794s 108ms/step - loss: 0.4040 - accuracy: 0.8531 - val_loss: 0.4137 - val_accuracy: 0.8493
Epoch 7/300
16685/16685 [==============================] - 1840s 110ms/step - loss: 0.39

# SMV2L lr=0.008

In [20]:
n_classes=62
smv2_l_008=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_008.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [21]:
opt=tf.keras.optimizers.Adam(learning_rate=0.008)
smv2_l_008.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [22]:
batch_size=256

In [23]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_008_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_008_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_008.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
2086/2086 [==============================] - 88s 39ms/step - loss: 0.6843 - accuracy: 0.7820 - val_loss: 0.5319 - val_accuracy: 0.8170
Epoch 2/300
2086/2086 [==============================] - 80s 38ms/step - loss: 0.4884 - accuracy: 0.8296 - val_loss: 0.5633 - val_accuracy: 0.8141
Epoch 3/300
2086/2086 [==============================] - 76s 37ms/step - loss: 0.4851 - accuracy: 0.8304 - val_loss: 0.5374 - val_accuracy: 0.8028
Epoch 4/300
2086/2086 [==============================] - 81s 39ms/step - loss: 0.4920 - accuracy: 0.8284 - val_loss: 0.6572 - val_accuracy: 0.7816
Epoch 5/300
2086/2086 [==============================] - 79s 38ms/step - loss: 0.5098 - accuracy: 0.8236 - val_loss: 0.8130 - val_accuracy: 0.7368
Epoch 6/300
2086/2086 [==============================] - 79s 38ms/step - loss: 0.5347 - accuracy: 0.8162 - val_loss: 0.8057 - val_accuracy: 0.7436
Epoch 7/300
2086/2086 [==============================] - 79s 38ms/step - loss: 0.5878 - accuracy: 0.8012 - val_loss: 1

# SMV2L lr=0.004

In [24]:
n_classes=62
smv2_l_004=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_004.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_6 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [25]:
opt=tf.keras.optimizers.Adam(learning_rate=0.004)
smv2_l_004.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [26]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_004_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_004_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_004.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
2086/2086 [==============================] - 81s 38ms/step - loss: 0.6982 - accuracy: 0.7777 - val_loss: 0.5824 - val_accuracy: 0.8058
Epoch 2/300
2086/2086 [==============================] - 80s 39ms/step - loss: 0.4517 - accuracy: 0.8398 - val_loss: 0.5601 - val_accuracy: 0.8122
Epoch 3/300
2086/2086 [==============================] - 81s 39ms/step - loss: 0.4322 - accuracy: 0.8446 - val_loss: 0.5021 - val_accuracy: 0.8272
Epoch 4/300
2086/2086 [==============================] - 82s 39ms/step - loss: 0.4191 - accuracy: 0.8486 - val_loss: 0.4730 - val_accuracy: 0.8305
Epoch 5/300
2086/2086 [==============================] - 82s 39ms/step - loss: 0.4068 - accuracy: 0.8527 - val_loss: 0.4143 - val_accuracy: 0.8509
Epoch 6/300
2086/2086 [==============================] - 81s 39ms/step - loss: 0.3962 - accuracy: 0.8554 - val_loss: 0.4149 - val_accuracy: 0.8486
Epoch 7/300
2086/2086 [==============================] - 81s 39ms/step - loss: 0.3865 - accuracy: 0.8581 - val_loss: 0

# SMV2L lr=0.003

In [27]:
n_classes=62
smv2_l_003=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_003.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_8 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [28]:
opt=tf.keras.optimizers.Adam(learning_rate=0.003)
smv2_l_003.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [29]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_003_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_003_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_003.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
2086/2086 [==============================] - 84s 38ms/step - loss: 0.7377 - accuracy: 0.7677 - val_loss: 0.5027 - val_accuracy: 0.8276
Epoch 2/300
2086/2086 [==============================] - 78s 37ms/step - loss: 0.4465 - accuracy: 0.8406 - val_loss: 0.4940 - val_accuracy: 0.8282
Epoch 3/300
2086/2086 [==============================] - 77s 37ms/step - loss: 0.4235 - accuracy: 0.8470 - val_loss: 0.4516 - val_accuracy: 0.8402
Epoch 4/300
2086/2086 [==============================] - 78s 37ms/step - loss: 0.4082 - accuracy: 0.8518 - val_loss: 0.4499 - val_accuracy: 0.8385
Epoch 5/300
2086/2086 [==============================] - 77s 37ms/step - loss: 0.3949 - accuracy: 0.8552 - val_loss: 0.4250 - val_accuracy: 0.8480
Epoch 6/300
2086/2086 [==============================] - 79s 38ms/step - loss: 0.3839 - accuracy: 0.8584 - val_loss: 0.4014 - val_accuracy: 0.8508
Epoch 7/300
2086/2086 [==============================] - 77s 37ms/step - loss: 0.3750 - accuracy: 0.8612 - val_loss: 0

# SMV2L lr=0.002

In [30]:
n_classes=62
smv2_l_002=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_002.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_10 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [31]:
opt=tf.keras.optimizers.Adam(learning_rate=0.002)
smv2_l_002.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [32]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_002_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_002_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_002.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
2086/2086 [==============================] - 82s 38ms/step - loss: 0.7577 - accuracy: 0.7610 - val_loss: 0.4967 - val_accuracy: 0.8254
Epoch 2/300
2086/2086 [==============================] - 79s 38ms/step - loss: 0.4472 - accuracy: 0.8404 - val_loss: 0.4757 - val_accuracy: 0.8356
Epoch 3/300
2086/2086 [==============================] - 79s 38ms/step - loss: 0.4218 - accuracy: 0.8474 - val_loss: 0.4432 - val_accuracy: 0.8426
Epoch 4/300
2086/2086 [==============================] - 78s 38ms/step - loss: 0.4011 - accuracy: 0.8535 - val_loss: 0.4128 - val_accuracy: 0.8506
Epoch 5/300
2086/2086 [==============================] - 77s 37ms/step - loss: 0.3873 - accuracy: 0.8571 - val_loss: 0.4110 - val_accuracy: 0.8513
Epoch 6/300
2086/2086 [==============================] - 75s 36ms/step - loss: 0.3764 - accuracy: 0.8602 - val_loss: 0.3974 - val_accuracy: 0.8507
Epoch 7/300
2086/2086 [==============================] - 75s 36ms/step - loss: 0.3662 - accuracy: 0.8633 - val_loss: 0

# SMV2L lr=0.0005

In [33]:
n_classes=62
smv2_l_0005=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_0005.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_12 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [34]:
opt=tf.keras.optimizers.Adam(learning_rate=0.0005)
smv2_l_0005.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [35]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_0005_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_0005_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_0005.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
2086/2086 [==============================] - 89s 41ms/step - loss: 1.1592 - accuracy: 0.6573 - val_loss: 0.5807 - val_accuracy: 0.7985
Epoch 2/300
2086/2086 [==============================] - 81s 39ms/step - loss: 0.5064 - accuracy: 0.8217 - val_loss: 0.4744 - val_accuracy: 0.8333
Epoch 3/300
2086/2086 [==============================] - 79s 38ms/step - loss: 0.4378 - accuracy: 0.8416 - val_loss: 0.4343 - val_accuracy: 0.8447
Epoch 4/300
2086/2086 [==============================] - 78s 37ms/step - loss: 0.4050 - accuracy: 0.8512 - val_loss: 0.4129 - val_accuracy: 0.8492
Epoch 5/300
2086/2086 [==============================] - 76s 37ms/step - loss: 0.3845 - accuracy: 0.8566 - val_loss: 0.4041 - val_accuracy: 0.8543
Epoch 6/300
2086/2086 [==============================] - 78s 37ms/step - loss: 0.3698 - accuracy: 0.8613 - val_loss: 0.3955 - val_accuracy: 0.8540
Epoch 7/300
2086/2086 [==============================] - 78s 37ms/step - loss: 0.3573 - accuracy: 0.8652 - val_loss: 0

# SMV2L lr=0.0001

In [36]:
n_classes=62
smv2_l_0001=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_0001.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_14 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [37]:
opt=tf.keras.optimizers.Adam(learning_rate=0.0001)
smv2_l_0001.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [38]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_0001_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_0001_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_0001.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
2086/2086 [==============================] - 80s 37ms/step - loss: 2.5064 - accuracy: 0.3514 - val_loss: 1.3740 - val_accuracy: 0.5896
Epoch 2/300
2086/2086 [==============================] - 77s 37ms/step - loss: 0.9975 - accuracy: 0.6874 - val_loss: 0.7513 - val_accuracy: 0.7533
Epoch 3/300
2086/2086 [==============================] - 76s 36ms/step - loss: 0.6429 - accuracy: 0.7839 - val_loss: 0.5674 - val_accuracy: 0.8053
Epoch 4/300
2086/2086 [==============================] - 77s 37ms/step - loss: 0.5176 - accuracy: 0.8192 - val_loss: 0.4931 - val_accuracy: 0.8267
Epoch 5/300
2086/2086 [==============================] - 76s 37ms/step - loss: 0.4571 - accuracy: 0.8366 - val_loss: 0.4619 - val_accuracy: 0.8365
Epoch 6/300
2086/2086 [==============================] - 76s 37ms/step - loss: 0.4210 - accuracy: 0.8472 - val_loss: 0.4429 - val_accuracy: 0.8408
Epoch 7/300
2086/2086 [==============================] - 77s 37ms/step - loss: 0.3959 - accuracy: 0.8541 - val_loss: 0

# SMV2L lr=0.005

In [48]:
n_classes=62
smv2_l_005=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_005.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_22 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_11 (Conv2D)          (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)  

In [49]:
opt=tf.keras.optimizers.Adam(learning_rate=0.005)
smv2_l_005.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [50]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_005_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_005_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_005.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
2086/2086 [==============================] - 96s 44ms/step - loss: 0.6935 - accuracy: 0.7801 - val_loss: 0.4911 - val_accuracy: 0.8292
Epoch 2/300
2086/2086 [==============================] - 84s 40ms/step - loss: 0.4586 - accuracy: 0.8379 - val_loss: 0.5821 - val_accuracy: 0.7981
Epoch 3/300
2086/2086 [==============================] - 93s 44ms/step - loss: 0.4433 - accuracy: 0.8413 - val_loss: 0.4945 - val_accuracy: 0.8316
Epoch 4/300
2086/2086 [==============================] - 89s 43ms/step - loss: 0.4292 - accuracy: 0.8456 - val_loss: 0.4405 - val_accuracy: 0.8416
Epoch 5/300
2086/2086 [==============================] - 85s 41ms/step - loss: 0.4198 - accuracy: 0.8482 - val_loss: 0.4594 - val_accuracy: 0.8355
Epoch 6/300
2086/2086 [==============================] - 82s 39ms/step - loss: 0.4100 - accuracy: 0.8513 - val_loss: 0.4737 - val_accuracy: 0.8326
Epoch 7/300
2086/2086 [==============================] - 83s 40ms/step - loss: 0.4029 - accuracy: 0.8531 - val_loss: 0

In [51]:
batch_size=128

In [52]:
n_classes=62
smv2_l_002_128=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_002_128.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_24 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_12 (Conv2D)          (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)  

In [53]:
opt=tf.keras.optimizers.Adam(learning_rate=0.002)
smv2_l_002_128.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [54]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_002_128_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_005_128_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_002_128.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
4172/4172 [==============================] - 154s 36ms/step - loss: 0.7645 - accuracy: 0.7598 - val_loss: 0.5188 - val_accuracy: 0.8225
Epoch 2/300
4172/4172 [==============================] - 141s 34ms/step - loss: 0.4644 - accuracy: 0.8353 - val_loss: 0.5044 - val_accuracy: 0.8260
Epoch 3/300
4172/4172 [==============================] - 139s 33ms/step - loss: 0.4281 - accuracy: 0.8455 - val_loss: 0.4324 - val_accuracy: 0.8459
Epoch 4/300
4172/4172 [==============================] - 139s 33ms/step - loss: 0.4075 - accuracy: 0.8516 - val_loss: 0.4170 - val_accuracy: 0.8487
Epoch 5/300
4172/4172 [==============================] - 140s 34ms/step - loss: 0.3926 - accuracy: 0.8557 - val_loss: 0.3934 - val_accuracy: 0.8571
Epoch 6/300
4172/4172 [==============================] - 138s 33ms/step - loss: 0.3824 - accuracy: 0.8591 - val_loss: 0.3931 - val_accuracy: 0.8574
Epoch 7/300
4172/4172 [==============================] - 140s 33ms/step - loss: 0.3747 - accuracy: 0.8612 - val_

In [55]:
n_classes=62
smv2_l_004_128=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[32,32,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l_004_128.summary()

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_26 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_13 (Conv2D)          (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)  

In [56]:
opt=tf.keras.optimizers.Adam(learning_rate=0.004)
smv2_l_004_128.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [57]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_adam_lr_004_128_recent%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_lr_004_128_recent.h5",save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history=smv2_l_004_128.fit(X_train_resize, y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize, y_valid),
                    epochs=300, callbacks=[tboard_callback, check_cb, earlystop_cb])

Epoch 1/300
4172/4172 [==============================] - 158s 37ms/step - loss: 0.7059 - accuracy: 0.7754 - val_loss: 0.5770 - val_accuracy: 0.8048
Epoch 2/300
4172/4172 [==============================] - 141s 34ms/step - loss: 0.4905 - accuracy: 0.8288 - val_loss: 0.5445 - val_accuracy: 0.8056
Epoch 3/300
4172/4172 [==============================] - 141s 34ms/step - loss: 0.4634 - accuracy: 0.8359 - val_loss: 0.4785 - val_accuracy: 0.8302
Epoch 4/300
4172/4172 [==============================] - 144s 35ms/step - loss: 0.4449 - accuracy: 0.8414 - val_loss: 0.5012 - val_accuracy: 0.8246
Epoch 5/300
4172/4172 [==============================] - 142s 34ms/step - loss: 0.4359 - accuracy: 0.8438 - val_loss: 0.5636 - val_accuracy: 0.8099
Epoch 6/300
4172/4172 [==============================] - 143s 34ms/step - loss: 0.4300 - accuracy: 0.8460 - val_loss: 0.5409 - val_accuracy: 0.8170
Epoch 7/300
4172/4172 [==============================] - 143s 34ms/step - loss: 0.4281 - accuracy: 0.8464 - val_